In [1]:
from src import host
from src import common
from src.setup import *

# Introduction

As an non-interactive usage of this notebook, one can use `sbatch src/run_gwas.sh` to send a job and get the results in a notebook or in HTML format. 

**TODO**: 

# README !

This notebook is special: it's designed differently from the others. To use this notebook effciently and interactively, first use `sbatch src/run_gwas.sh` to run a job that...

###### A word about `run_gwas()` function

Every output is stored. Warnings are displayed in `concatenated_warnings`, and the whole reports in `concatenated_logs` (both paths are defined in `setup.py`).

# Compute assciation tests



In [2]:
#common.write_phenotypes(PATH_ASIANS_GWAS, phenotype='all', output_path=PATH_ASIANS_GWAS_PHENOTYPES)
#host.run_gwas()

# Check the results

###### Report computational time


In [3]:
!cat $PATH_ASIANS_GWAS_TMP_LOGS_CONCAT | grep "Start time:" | head -1 
!cat $PATH_ASIANS_GWAS_TMP_LOGS_CONCAT | grep "End time:" | tail -1 

Start time: Tue Aug 20 18:20:28 2019
grep: erreur d'écriture
cat: erreur d'écriture: Relais brisé (pipe)
End time: Tue Aug 20 18:38:07 2019


This took about 18 min 30 s.

###### Look at warnings



In [4]:
# count warnings
print("Number of warnings: ", end='')
!grep $PATH_ASIANS_GWAS_TMP_LOGS_CONCAT -e Warning | wc -l

Number of warnings: 633


In [5]:
!head $PATH_ASIANS_GWAS_TMP_WARNINGS


(PC_9_V) Warning: --glm remaining case count is less than 10x predictor count for
phenotype 'PC_9_V'.

(PC_28_STOP) 
(PC_28_W) Warning: --glm remaining control count is less than 10x predictor count for
phenotype 'PC_28_W'.

(PC_29_D) Warning: --glm remaining case count is less than 10x predictor count for
phenotype 'PC_29_D'.


We have 633/744 amino acids that do not respect the 1 to 10 rule of the thumb.

###### Skipped amino acids


In [6]:
!grep $PATH_ASIANS_GWAS_TMP_WARNINGS -e Skipping | wc -l

37


We have 37 amino acids that were skipped with the following warning:

    (S_19_P) Warning: Skipping --glm regression on phenotype <AA> since # samples <= #
    predictor columns.
    (Quasi-)separated covariate(s) were present.  Try removing inappropriate
    covariates, and/or using Firth logistic regression.

# Summarize results

###### p-values

In [7]:
# 1. Get all files in the temporary results directory
from os import walk
f = []
for (dirpath, dirnames, filenames) in walk(PATH_ASIANS_GWAS_TMP):
    f = filenames    

In [8]:
threshold = 5e-8

In [9]:
# 2. For each plink report file, get p-values and summarize them
summaries = pd.DataFrame()
concat = pd.DataFrame()

report = [1, 10, 50 100, 200, 300, 400, 500, 600, 700] # track the job

for counter, i in enumerate(f):
    if i[-9:] != '.logistic': continue 
    print('.', end='')
    if counter in report:
        !echo $counter
    
    df = pd.read_csv(PATH_ASIANS_GWAS_TMP+i, sep='\t')
    summary = df.P.describe()
    summary.name = i.split('.')[1]
    if summaries.empty: 
        summaries = pd.DataFrame(summary)
        concat = pd.DataFrame( df[ df.P <= threshold ] )
    else: 
        summaries.insert(loc=0, column=i.split('.')[1], value=summary)
        concat = concat.append(df[df.P <= threshold], ignore_index=True)
print()

..


In [10]:
common.write_binary(PATH_ASIANS_GWAS_TMP_SUMMARY, summaries)
common.write_binary(PATH_ASIANS_GWAS_FILTERED, concat)

write_binary(): successfully written in '/scratch/mazeller/gwas_tmp_results/pval_summary'
write_binary(): successfully written in 'data/plink/gwas/filtered_g2g'
